In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model="llama2")
embeddings = OllamaEmbeddings()

model.invoke("Tell me a joke")

/var/folders/hx/0ptq79g959sftkzv1vp9y9tm0000gn/T/ipykernel_68890/3784052260.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="llama2")
/var/folders/hx/0ptq79g959sftkzv1vp9y9tm0000gn/T/ipykernel_68890/3784052260.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings()


"Sure! Here's a classic one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!"

In [17]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
 
chain = model | parser
chain.invoke("tell me a joke")

"Sure! Here's a quick one:\n\nWhy don't scientists trust atoms? Because they make up everything!\n\nI hope that made you smile!"

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/Virtual_Companion_FAQs.pdf")
pages = loader.load_and_split()
pages

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 75 0 (offset 0)


[Document(metadata={'source': 'data/Virtual_Companion_FAQs.pdf', 'page': 0}, page_content="1. What should I include in the care packages for homeless individuals? Stick with basic essentials such as: ● Soap or hygiene products (avoid alcohol-based products). ● Socks (a highly needed item). ● Snacks or non-perishable food. ● Water bottles. ● Feminine hygiene products. ● First aid items (like bandages and antiseptic wipes, but avoid alcohol-based products). 2. Are there specific items that a re easier for homeless individuals to carry? Yes, individual bags with compact, lightweight items are easier to carry around. Avoid bulky or heavy items. 3. How should I distribute wellness information or resources? Carry printed Street Care wellness information and offer them when giving out care packages. These materials can provide additional help to those in need. 4. Should I offer help to everyone I meet? It's always good to offer assistance, but be mindful and respectful if someone declines hel

In [19]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below in a consice and precise manner in maximum 150 words. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)


print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below in a consice and precise manner in maximum 150 words. If you can't answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



In [20]:
chain = prompt | model | parser

In [21]:
chain.invoke(
    {
        "context": "The name I was given was Eli",
        'question': "What's my name?"
    }
)

' Of course! Based on the context provided, your name is Eli.'

In [22]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

DocArray is a vector storage, it is smaller as compared to Pincone and other vector databases.

vectorstore.as_retriever().invoke("homeless")

In [23]:
retriver = vectorstore.as_retriever()

retriver.invoke("homeless")

[Document(metadata={'source': 'Virtual_Companion_FAQs.pdf', 'page': 1}, page_content="It’s completely normal to feel overwhelmed. Take your time, and remember that even small actions can make a big impact. The experience often becomes easier with practice, and you may find inspiration along the way. 2. Can my help really make a difference? Absolutely! Your assistance, whether through providing essentials or simply offering support, can literally save lives by addressing immediate needs such as food, water, and first aid. 3. How do I stay motivated as a volunteer? The act of going out and helping others is often inspiring in itself. Seeing the direct impact of your efforts and connecting with those you help can be incredibly rewarding. 4. What should I do if I start to feel discouraged? It’s important to remind yourself that every small contribution counts. Reach out to other volunteers for support or share your experiences—it helps to know you're not alone in your efforts. 5. Is it oka

In [24]:
from operator import itemgetter

chain = (

    {
        "context": itemgetter("question") | retriver,
        "question": itemgetter("question")
    }
    | prompt
    | model
    | parser
)

chain.invoke({"question": "How can we show respect to homeless people?"})

" Based on the provided FAQ document, showing respect to homeless individuals involves several aspects:\n\n1. Comfortable clothing: Dress in casual, comfortable attire that won't make them feel uncomfortable or self-conscious during interactions. Avoid formal or work-related clothing that may intimidate them.\n2. Personal space: Maintain a distance of six feet between outreach peers and homeless individuals to respect their personal space.\n3. Hygiene: Wash hands with soap and water for 20 seconds or use hand sanitizer to maintain good hygiene practices.\n4. Avoiding assumptions: Keep an open mind when engaging with someone experiencing homelessness, avoiding assumptions based on appearance or behavior. Ask questions and listen carefully to understand their experiences.\n5. Dress code: Avoid wearing uniforms or formal clothing that may trigger anxiety or discomfort in people experiencing homelessness. Instead, wear casual attire that appears welcoming and non-threatening.\n\nBy followi

In [25]:
output = chain.invoke({"question": "How can we show respect to homeless people?"})  # Chain invocation with input data

# Clean the output text to remove newlines
cleaned_output = output.replace('\n', ' ').strip()  # Replaces \n with a space and strips leading/trailing whitespace

print(cleaned_output)

Based on the provided FAQ document, here is how you can show respect to homeless people:  1. Dress comfortably and avoid uniforms that may trigger anxiety or discomfort in the individuals you are helping. 2. Maintain a distance of six feet between outreach peers and homeless individuals to ensure safety. 3. Wear masks while out in public and wash your hands with soap and water for 20 seconds or use hand sanitizer to maintain personal hygiene. 4. Avoid touching your face to prevent potential contamination.  By following these guidelines, you can show respect to homeless individuals by prioritizing their safety and well-being.


In [26]:
questions = [
    "What should I do if I start to feel discouraged?",
    "What should I include in the care packages for homeless individuals?",
    "What are the benefits of working in a group?",
    "What if I feel nervous during the interaction?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({"question": question})}")
    print()

Question: What should I do if I start to feel discouraged?
Answer:  If you start to feel discouraged while interacting with homeless individuals, it's important to take a step back and prioritize your own well-being. Remember that showing respect and empathy towards others doesn't mean neglecting yourself. Here are some suggestions:

1. Take a break: If you're feeling overwhelmed or discouraged, take a short break to rest and recharge. This can help you approach the interaction with renewed energy and positivity.
2. Reflect on your intentions: Remind yourself why you're engaging with homeless individuals in the first place. Focus on the purpose of showing respect and empathy, rather than feeling discouraged by individual situations.
3. Practice self-care: Engage in activities that help you relax and take care of yourself, such as exercise, meditation, or spending time with loved ones. This can help you maintain a positive mindset when interacting with homeless individuals.
4. Seek supp

In [27]:
chain.batch([{"question":q} for q in questions])

[" To answer the question, it is important to recognize that feeling discouraged is a normal part of the process when working with others to enhance knowledge. Here are some possible actions you can take to address this feeling:\n\n1. Take a break: If you're feeling overwhelmed or discouraged, taking a break can help you recharge and come back to the task with renewed energy and focus.\n2. Reflect on progress: Take time to reflect on the progress you've made so far. Celebrate your achievements, no matter how small they may seem. This can help boost your morale and motivation.\n3. Seek support: Don't be afraid to seek support from colleagues, mentors, or peers. Talking through challenges with someone who understands can provide valuable perspective and encouragement.\n4. Reframe negative thoughts: Challenge any negative self-talk that may be contributing to your discouragement. Replace these thoughts with more positive and realistic ones.\n5. Find inspiration: Surround yourself with sou